In [ ]:
%matplotlib inline

import os
import pickle
from copy import deepcopy

import iris
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import ndimage as nd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from wildfires.analysis.plotting import partial_dependence_plot
from wildfires.data.cube_aggregation import aggregate_cubes
from wildfires.data.datasets import DATA_DIR, data_map_plot

In [ ]:
aggregate_cubes()

In [ ]:
import warnings

warnings.filterwarnings("ignore", ".*Collapsing a non-contiguous coordinate.*")

In [ ]:
def log_mapping(key):
    if "log" in key:
        return False
    if key.lower() in {"monthly burned area", "popd"}:
        return True
    if " ".join(key.lower().split(" ")[1:]) in {"monthly burned area", "popd"}:
        return True
    return False

In [ ]:
with open(os.path.join(DATA_DIR, "monthly_climatologies.pickle"), "rb") as f:
    mean_cubes = pickle.load(f)
print(mean_cubes)
# so that analysis below can be replicated for other kinds of cubes
cubes = mean_cubes

In [ ]:
# Get land mask from the data
mpl.rcParams["figure.figsize"] = (8, 5)

land_cube = cubes.extract_strict(iris.Constraint(name="pftNoLand"))
land_mask = np.isclose(land_cube.data.data, 1.0)
fig = data_map_plot(np.ma.mean(land_mask, axis=0), name="Land Mask")

# remove this cube from the CubeList
del cubes[cubes.index(land_cube)]

In [ ]:
# Define a latitude mask which ignores data beyond 60 degrees, as the precipitation data has strange artifacts at those latitudes.
lats = cubes[0].coord("latitude").points
lons = cubes[1].coord("longitude").points
lat_mask = (
    np.zeros_like(land_mask) + (np.meshgrid(lats, lons, indexing="ij")[0] > 60)[None]
)

lat_land_cubes = deepcopy(cubes)

for cube in lat_land_cubes:
    cube.data.mask[lat_mask] = True
    cube.data.mask[land_mask] = True

n_cols = 4
n_plots = len(lat_land_cubes)

mpl.rcParams["figure.figsize"] = (20, 12)

fig, axes = plt.subplots(
    nrows=int(np.ceil(float(n_plots) / n_cols)), ncols=n_cols, squeeze=False
)
axes = axes.flatten()
for (i, (ax, feature)) in enumerate(zip(axes, range(n_plots))):
    ax.hist(
        lat_land_cubes[feature].data.data[~lat_land_cubes[feature].data.mask],
        density=True,
        bins=70,
    )
    ax.set_xlabel(lat_land_cubes[feature].name())
    ax.set_yscale("log")

for ax in axes[n_plots:]:
    ax.set_axis_off()

plt.tight_layout()

In [ ]:
# Raw Data
mpl.rcParams["figure.figsize"] = (10, 7)
for cube in lat_land_cubes:
    fig = data_map_plot(
        cube.collapsed("time", iris.analysis.MEAN), log=log_mapping(cube.name())
    )

In [ ]:
# Check that all the cubes have masks
assert np.all([hasattr(cube.data, "mask") for cube in cubes])
# Respect the masking of 'monthly burned area' and ignore all others - for all others, replace
# masked data using nearest-neighbour interpolation.
# Thereafter, apply the land_mask and lat_mask, so that only data over land and within the latitude limits is considered.
# Latitude limits due to anomalous behaviour of precipitation data, as well as limitations of the lightning LIS/OTD dataset.

burned_area_cube = cubes.extract_strict(iris.Constraint(name="monthly burned area"))
burned_area_mask = burned_area_cube.data.mask
combined_mask = burned_area_mask | land_mask | lat_mask

cubes_mod = deepcopy(cubes)
assert isinstance(cubes_mod, iris.cube.CubeList)
datasets_botched = 0
for cube in cubes_mod:
    # In this part, data gaps are filled, so that the maximum possible area of data (limited by where burned area data is available)
    # is used for the analysis.
    # Choose to fill the gaps using nearest-neighbour interpolation.
    # To do this, define a mask which will tell the algorithm where to replace data.

    # Ignore burned area in this step, as this should never be modified!
    if cube.name() != "monthly burned area":
        print(cube.name())
        # Replace data where it is masked.
        fill_mask = cube.data.mask

        # Additional data replacing for datasets below.
        if cube.name() == "SIF":
            # Replace where it is above 20.
            fill_mask |= cube.data.data > 20
            # Replace where it is below 0.
            fill_mask |= cube.data.data < 0
            datasets_botched += 1
        elif cube.name() == "Combined Flash Rate Time Series":
            # Replace where it is below 0.
            fill_mask |= cube.data.data < 0
            datasets_botched += 1

        orig_data = cube.data.data.copy()
        if np.any(fill_mask):
            print(
                "Filling {:} elements ({:} after final masking).".format(
                    np.sum(fill_mask), np.sum(fill_mask[~combined_mask])
                )
            )
            filled_data = cube.data.data[
                tuple(
                    nd.distance_transform_edt(
                        fill_mask, return_distances=False, return_indices=True
                    )
                )
            ]
            assert np.all(np.isclose(cube.data.data[~fill_mask], orig_data[~fill_mask]))

            selected_unfilled_data = orig_data[~combined_mask]
            selected_filled_data = filled_data[~combined_mask]

            print(
                "Min {:0.1e}/{:0.1e}, max {:0.1e}/{:0.1e} before/after filling (for relevant regions)".format(
                    np.min(selected_unfilled_data),
                    np.min(selected_filled_data),
                    np.max(selected_unfilled_data),
                    np.max(selected_filled_data),
                )
            )
        else:
            # Prevent overwriting with previous loop's filled data.
            filled_data = orig_data

        # Always apply global combined mask.
        cube.data = np.ma.MaskedArray(filled_data, mask=combined_mask)
        print("")
    else:
        # Always apply global combined mask.
        cube.data.mask = combined_mask

# Assert that both SIF and Flash Rates were handled.
assert datasets_botched == 2

# Check that there aren't any inf's or nan's in the data.
for cube in cubes_mod:
    assert not np.any(np.isinf(cube.data.data[~cube.data.mask]))
    assert not np.any(np.isnan(cube.data.data[~cube.data.mask]))

In [ ]:
mpl.rcParams["figure.figsize"] = (10, 7)
for cube in cubes_mod:
    fig = data_map_plot(
        cube.collapsed("time", iris.analysis.MEAN), log=log_mapping(cube.name())
    )

In [ ]:
n_cols = 4
n_plots = len(cubes_mod)

mpl.rcParams["figure.figsize"] = (20, 12)

fig, axes = plt.subplots(
    nrows=int(np.ceil(float(n_plots) / n_cols)), ncols=n_cols, squeeze=False
)
axes = axes.flatten()
for (i, (ax, feature)) in enumerate(zip(axes, range(n_plots))):
    ax.hist(
        cubes_mod[feature].data.data[~cubes_mod[feature].data.mask],
        density=True,
        bins=70,
    )
    ax.set_xlabel(cubes_mod[feature].name())
    ax.set_yscale("log")

for ax in axes[n_plots:]:
    ax.set_axis_off()

plt.tight_layout()

In [ ]:
burned_area_cube = cubes_mod.extract_strict(iris.Constraint(name="monthly burned area"))
endog_data = pd.Series(burned_area_cube.data.data[~burned_area_cube.data.mask])
names = []
data = []
for cube in cubes_mod:
    if cube.name() != "monthly burned area":
        names.append(cube.name())
        data.append(cube.data.data[~cube.data.mask].reshape(-1, 1))
exog_data = pd.DataFrame(np.hstack(data), columns=names)
exog_data["temperature range"] = (
    exog_data["maximum temperature"] - exog_data["minimum temperature"]
)
del exog_data["minimum temperature"]

print(names)

# Carry out log transformation for select variables.
log_var_names = ["temperature range", "dry_days", "dry_day_period"]

for name in log_var_names:
    mod_data = exog_data[name] + 0.01
    assert np.all(mod_data >= (0.01 - 1e-8)), "{:}".format(name)
    exog_data["log " + name] = np.log(mod_data)
    del exog_data[name]

# Carry out square root transformation
sqrt_var_names = ["Combined Flash Rate Time Series", "popd"]
for name in sqrt_var_names:
    assert np.all(exog_data[name] >= 0), "{:}".format(name)
    exog_data["sqrt " + name] = np.sqrt(exog_data[name])
    del exog_data[name]

In [ ]:
model = sm.GLM(endog_data, exog_data, family=sm.families.Binomial())
model_results = model.fit()
print(model_results.summary())

mpl.rcParams["figure.figsize"] = (12, 9)

plt.figure()
plt.hexbin(endog_data, model_results.fittedvalues, bins="log")
plt.xlabel("real data")
plt.ylabel("prediction")
plt.colorbar()

global_mask = burned_area_cube.data.mask

ba_predicted = np.zeros_like(global_mask, dtype=np.float64)
ba_predicted[~global_mask] = model_results.fittedvalues
ba_predicted = np.ma.MaskedArray(ba_predicted, mask=global_mask)
fig = data_map_plot(
    np.ma.mean(ba_predicted, axis=0), name="Predicted Mean Burned Area", log=True
)

ba_data = np.zeros_like(global_mask, dtype=np.float64)
ba_data[~global_mask] = endog_data.values
ba_data = np.ma.MaskedArray(ba_data, mask=global_mask)
fig = data_map_plot(
    np.ma.mean(ba_data, axis=0), name="Mean observed burned area (GFEDv4)", log=True
)

_ = plt.matshow(exog_data.corr())
_ = plt.xticks(range(len(exog_data.columns)), exog_data.columns, rotation="vertical")
_ = plt.yticks(range(len(exog_data.columns)), exog_data.columns)
_ = plt.colorbar()

print("R2:", r2_score(y_true=endog_data, y_pred=model_results.fittedvalues))

In [ ]:
# Remove redundant variables: Soil Water Index, LAI, precip
exog_data2 = deepcopy(exog_data)
del exog_data2["Soil Water Index with T=1"]
del exog_data2["Leaf Area Index"]
del exog_data2["precip"]

model = sm.GLM(endog_data, exog_data2, family=sm.families.Binomial())
model_results = model.fit()
print(model_results.summary())

mpl.rcParams["figure.figsize"] = (12, 9)

plt.figure()
plt.hexbin(endog_data, model_results.fittedvalues, bins="log")
plt.xlabel("real data")
plt.ylabel("prediction")
plt.colorbar()

global_mask = burned_area_cube.data.mask

ba_predicted = np.zeros_like(global_mask, dtype=np.float64)
ba_predicted[~global_mask] = model_results.fittedvalues
ba_predicted = np.ma.MaskedArray(ba_predicted, mask=global_mask)
fig = data_map_plot(
    np.ma.mean(ba_predicted, axis=0), name="Predicted Mean Burned Area", log=True
)

ba_data = np.zeros_like(global_mask, dtype=np.float64)
ba_data[~global_mask] = endog_data.values
ba_data = np.ma.MaskedArray(ba_data, mask=global_mask)
fig = data_map_plot(
    np.ma.mean(ba_data, axis=0), name="Mean observed burned area (GFEDv4)", log=True
)

columns = exog_data2.columns
_ = plt.matshow(exog_data2.corr())
_ = plt.xticks(range(len(columns)), columns, rotation="vertical")
_ = plt.yticks(range(len(columns)), columns)
_ = plt.colorbar()

print("R2:", r2_score(y_true=endog_data, y_pred=model_results.fittedvalues))

In [ ]:
regr = RandomForestRegressor(n_estimators=50, random_state=1, n_jobs=4)
X_train, X_test, y_train, y_test = train_test_split(
    exog_data2, endog_data, random_state=1, shuffle=True, test_size=0.3
)
regr.fit(X_train, y_train)
print("R2 train:", regr.score(X_train, y_train))
print("R2 test:", regr.score(X_test, y_test))

In [ ]:
mpl.rcParams["figure.figsize"] = (20, 12)
from importlib import reload

import wildfires.analysis.plotting

reload(wildfires.analysis.plotting)
fig, axes = wildfires.analysis.plotting.partial_dependence_plot(
    regr,
    X_test,
    X_test.columns,
    n_cols=4,
    grid_resolution=50,
    coverage=0.02,
    random_state=1,
    predicted_name="burned area",
)

In [ ]:
mpl.rcParams["figure.figsize"] = (12, 9)

plt.figure()
plt.hexbin(
    y_test,
    regr.predict(X_test),
    bins="log",
    # xscale='log', yscale='log'
)
plt.xlabel("real data")
plt.ylabel("prediction")
plt.colorbar()

mpl.rcParams["figure.figsize"] = (12, 10)

global_mask = burned_area_cube.data.mask

ba_predicted = np.zeros_like(global_mask, dtype=np.float64)
ba_predicted[~global_mask] = regr.predict(exog_data2)
ba_predicted = np.ma.MaskedArray(ba_predicted, mask=global_mask)
fig = data_map_plot(
    np.ma.mean(ba_predicted, axis=0), name="Predicted Mean Burned Area", log=True
)

ba_data = np.zeros_like(global_mask, dtype=np.float64)
ba_data[~global_mask] = endog_data
ba_data = np.ma.MaskedArray(ba_data, mask=global_mask)
fig = data_map_plot(
    np.ma.mean(ba_data, axis=0), name="Mean observed burned area (GFEDv4)", log=True
)

In [ ]:
mpl.rcParams["figure.figsize"] = (20, 12)

n_cols = 4

fig, axes = plt.subplots(
    nrows=int(np.ceil(float(len(exog_data2.columns)) / n_cols)),
    ncols=n_cols,
    squeeze=False,
)
axes = axes.flatten()
for (i, (ax, feature)) in enumerate(zip(axes, exog_data2.columns)):
    ax.hist(exog_data2[feature], density=True, bins=70)
    ax.set_xlabel(feature)
    ax.set_yscale("log")

for ax in axes[len(exog_data2.columns) :]:
    ax.set_axis_off()

plt.tight_layout()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

print("{:<59s} | {:<10s}".format("Name", "VIF"))
for i, name in enumerate(exog_data.columns):
    print(
        "{:<59s} | {:>10.1f}".format(
            name, variance_inflation_factor(exog_data.values, i)
        )
    )

In [ ]:
exog_data.values.shape

In [ ]:
print("{:<59s} | {:<10s}".format("Name", "VIF"))
for i, name in enumerate(exog_data.columns):
    X_fit = exog_data.values[:, i].reshape(-1, 1)
    X_k = exog_data.values[:, [j for j in range(len(exog_data.columns)) if j != i]]
    X_k = np.hstack((np.ones(X_k.shape[0]).reshape(-1, 1), X_k))

    x, res, rank, s = np.linalg.lstsq(X_k, X_fit, rcond=None)
    predicted = X_k.dot(x)

    vif = 1.0 / (1 - r2_score(y_true=X_fit, y_pred=predicted))

    # r2 = r2_score(y_true=X_fit, y_pred=predicted)
    # alt_r2 = OLS(X_fit, X_k).fit().rsquared
    # print('{:+>5.1e} {:+>5.1e}'.format(r2, alt_r2))
    # The r2 scores above do match, but the VIF values do not. This is likely due to
    # the fact that the statsmodels implementation uses the OLS linear fit procedure,
    # which does not add a constant by default (as was done explicitly above using
    # np.ones()). This is the only difference I could find, as the methods are otherwise
    # identical.
    print("{:<59s} | {:>10.1f}".format(name, vif))

In [ ]:
print("After variable removal")
print("{:<59s} | {:<10s}".format("Name", "VIF"))
for i, name in enumerate(exog_data2.columns):
    X_fit = exog_data2.values[:, i].reshape(-1, 1)
    X_k = exog_data2.values[:, [j for j in range(len(exog_data2.columns)) if j != i]]
    X_k = np.hstack((np.ones(X_k.shape[0]).reshape(-1, 1), X_k))

    x, res, rank, s = np.linalg.lstsq(X_k, X_fit, rcond=None)
    predicted = X_k.dot(x)

    vif = 1.0 / (1 - r2_score(y_true=X_fit, y_pred=predicted))

    # r2 = r2_score(y_true=X_fit, y_pred=predicted)
    # alt_r2 = OLS(X_fit, X_k).fit().rsquared
    # print('{:+>5.1e} {:+>5.1e}'.format(r2, alt_r2))
    # The r2 scores above do match, but the VIF values do not. This is likely due to
    # the fact that the statsmodels implementation uses the OLS linear fit procedure,
    # which does not add a constant by default (as was done explicitly above using
    # np.ones()). This is the only difference I could find, as the methods are otherwise
    # identical.
    print("{:<59s} | {:>10.1f}".format(name, vif))

In [ ]:
from statsmodels.regression.linear_model import OLS

i = 0

X_fit = exog_data.values[:, i].reshape(-1, 1)
X_k = exog_data.values[:, [j for j in range(len(exog_data.columns)) if j != i]]
X_k = np.hstack((np.ones(X_k.shape[0]).reshape(-1, 1), X_k))

x, res, rank, s = np.linalg.lstsq(X_k, X_fit, rcond=None)
predicted1 = X_k.dot(x)

alt_r2 = OLS(X_fit, X_k).fit().rsquared
results = model.fit()

In [ ]:
for i in range(x.shape[0]):
    print("{:0.1e} {:0.1e}".format(x.flatten()[i], results.params[i]))